In [1]:
import Plots; Plots.gr()

Plots.GRBackend()

In [ ]:
using GaussianMixtures, StatsBase, Distributions

## 1.模拟实验数据

假设一种细胞群体包含两类细胞，数量分别占细胞总数的70%和30%这两类细胞在某种可观察性如荧光强度是有区别的，第一类细胞该种性分布均值100，均方偏差20的正态分布，第二类细胞该性分布均值150，均方偏差35的正态分布

In [3]:
N = 20000
srand(10)
x = [100 + 20randn(7N÷10,1); 150 + 35randn(3N÷10,1)]
Plots.histogram(x)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 0 
 
 
 500 
 
 
 1000 
 
 
 1500 
 
 
 2000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

## 2. 极大似然参数估计

In [ ]:
g1 = GMM(1, x; method=:kmeans, kind=:diag, nInit=50, nIter=100, nFinal=10)
m1 = MixtureModel(g1)

In [5]:
for t = 1:3
    em!(g1, x)
    m1 = MixtureModel(g1)
    @show m1
end

m1 = MixtureModel{Distributions.Normal{Float64}}(K = 1)
components[1] (prior = 1.0000): Distributions.Normal{Float64}(μ=114.80025791788525, σ=34.08354257492853)

m1 = MixtureModel{Distributions.Normal{Float64}}(K = 1)
components[1] (prior = 1.0000): Distributions.Normal{Float64}(μ=114.80025791788525, σ=34.08354257492853)

m1 = MixtureModel{Distributions.Normal{Float64}}(K = 1)
components[1] (prior = 1.0000): Distributions.Normal{Float64}(μ=114.80025791788525, σ=34.08354257492853)



In [ ]:
g2 = GMM(2, x; method=:kmeans, kind=:diag, nInit=50, nIter=100, nFinal=10)
m2 = MixtureModel(g2)

In [7]:
for t = 1:3
    em!(g2, x)
    m2 = MixtureModel(g2)
    @show m2
end

m2 = MixtureModel{Distributions.Normal{Float64}}(K = 2)
components[1] (prior = 0.7045): Distributions.Normal{Float64}(μ=99.77748185832066, σ=20.01971220547362)
components[2] (prior = 0.2955): Distributions.Normal{Float64}(μ=150.6134334096199, σ=33.9855464810837)

m2 = MixtureModel{Distributions.Normal{Float64}}(K = 2)
components[1] (prior = 0.7030): Distributions.Normal{Float64}(μ=99.75483618916014, σ=20.002386938247103)
components[2] (prior = 0.2970): Distributions.Normal{Float64}(μ=150.41212868083255, σ=34.06381642737712)

m2 = MixtureModel{Distributions.Normal{Float64}}(K = 2)
components[1] (prior = 0.7017): Distributions.Normal{Float64}(μ=99.73483627968058, σ=19.986820678237272)
components[2] (prior = 0.2983): Distributions.Normal{Float64}(μ=150.23251709339817, σ=34.13339654528371)



In [ ]:
g3 = GMM(3, x; method=:kmeans, kind=:diag, nInit=50, nIter=100, nFinal=10)
m3 = MixtureModel(g3)

In [9]:
for t = 1:3
    em!(g3, x)
    m3 = MixtureModel(g3)
    @show m3
end

m3 = MixtureModel{Distributions.Normal{Float64}}(K = 3)
components[1] (prior = 0.2998): Distributions.Normal{Float64}(μ=110.33248235957015, σ=20.162007437151438)
components[2] (prior = 0.2495): Distributions.Normal{Float64}(μ=156.5637461063916, σ=32.012362515399346)
components[3] (prior = 0.4507): Distributions.Normal{Float64}(μ=94.65417720020324, σ=18.845463184917158)

m3 = MixtureModel{Distributions.Normal{Float64}}(K = 3)
components[1] (prior = 0.2976): Distributions.Normal{Float64}(μ=110.04541081193899, σ=20.19488983134557)
components[2] (prior = 0.2521): Distributions.Normal{Float64}(μ=156.17646863930813, σ=32.15765368167774)
components[3] (prior = 0.4503): Distributions.Normal{Float64}(μ=94.77417773794066, σ=18.885797154436897)

m3 = MixtureModel{Distributions.Normal{Float64}}(K = 3)
components[1] (prior = 0.2955): Distributions.Normal{Float64}(μ=109.77886933297866, σ=20.21745671957379)
components[2] (prior = 0.2547): Distributions.Normal{Float64}(μ=155.81283797003613, σ=32.29266

## 3.假设检验

把模拟实验结果作为观察数据，针对不同的N， 分别采用M1、M2作为null hypothesis, 用卡方检验，判别在显著性0.05时是否应该否定相应模型?


将数据x做直方图h，把直方图统计区间的最左边和最右边延展到无穷大，利用MixtureModel的pdf积分计算每个区间的理论概率 $E$，与实际直方图h得到的O结合计算卡方值 $\chi^{2}$, p是MixtureModel的混合数，length(x)-1-p是卡方分布的自由度，利用卡方分布的cdf可以计算significance

In [10]:
prob(m, x1, x2) = quadgk(x->pdf(m, x), x1, x2)[1]

prob (generic function with 1 method)

In [11]:
function Chisq_test(m, x)
    h = fit(Histogram, vec(x))
    edge = [-Inf; h.edges[1][2:end-1]; Inf]
    interval = [(edge[i], edge[i+1]) for i in 1:length(edge)-1]

    O = h.weights
    E = length(x)*[prob(m, ab...) for ab in interval]
    χ² = sum((O[i] - E[i]) ^ 2 / E[i] for i in 1:length(O))

    p = 2length(m.components)
    significance = 1 - cdf(Chisq(length(O) - 1 - p), χ²)
    significance
end

Chisq_test (generic function with 1 method)

In [12]:
@show significance = Chisq_test(m1, x)
@show significance = Chisq_test(m2, x);

significance = Chisq_test(m1,x) = 0.0
significance = Chisq_test(m2,x) = 0.29628740422799527


M2的显著性大于0.05， 不能否定M2

M1的显著性小于0.05， 否定M1

## 4. 模型选择

把模拟实验结果作为了观察数据，针对不同的N，用修正的AIC判据在M1， M2，M3 中做出选择

In [13]:
function AIC_select(m, x)
    k = 2length(m.components)
    L = sum(logpdf(m, x[i]) for i in 1:length(x))
    aic = 2k - 2L
    L, aic
end

AIC_select (generic function with 1 method)

L denotes likelihood

In [14]:
@show L, aic = AIC_select(m1, x)
@show L, aic = AIC_select(m2, x)
@show L, aic = AIC_select(m3, x);

(L,aic) = AIC_select(m1,x) = (-98955.06357116673,197914.12714233345)
(L,aic) = AIC_select(m2,x) = (-97266.23683571302,194540.47367142604)
(L,aic) = AIC_select(m3,x) = (-97268.72585880879,194549.45171761757)


M2的AIC最小， 选择M2